In [ ]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import re
from sklearn.preprocessing import LabelEncoder

# Load the data
data = pd.read_csv('/content/Sentiment (3) (1).csv')
data = data[['text','sentiment']]

# Preprocess the text data
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply(lambda x: re.sub('[^a-zA-z0-9\s]', '', x))
for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

# Tokenize and pad sequences
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

# Define the model architecture
embed_dim = 128
lstm_out = 196
def create_model():
    model = Sequential()
    model.add(Embedding(max_features, embed_dim, input_length=X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Encode the target labels
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Train the model
batch_size = 32
model = create_model()
model.fit(X_train, y_train, epochs=1, batch_size=batch_size, verbose=2)

# Evaluate the model
score, acc = model.evaluate(X_test, y_test, verbose=2, batch_size=batch_size)
print("Test score:", score)
print("Test accuracy:", acc)

# Save the model
model.save('sentiment_analysis_model.h5')

# Load the saved model
from keras.models import load_model
loaded_model = load_model('sentiment_analysis_model.h5')

# New text data for prediction
new_text = "A lot of good things are happening. We are respected again throughout the world, and that's a great thing."
new_text = new_text.lower()
new_text = re.sub('[^a-zA-z0-9\s]', '', new_text)
new_text_sequence = tokenizer.texts_to_sequences([new_text])
new_text_sequence = pad_sequences(new_text_sequence, maxlen=X.shape[1])

# Make predictions on new data
prediction = loaded_model.predict(new_text_sequence)
sentiment_labels = ['negative', 'neutral', 'positive']
predicted_sentiment = sentiment_labels[prediction.argmax()]

print("Predicted sentiment:", predicted_sentiment)


291/291 - 43s - loss: 0.8254 - accuracy: 0.6425 - 43s/epoch - 149ms/step
144/144 - 4s - loss: 0.7606 - accuracy: 0.6772 - 4s/epoch - 27ms/step
Test score: 0.7606064677238464
Test accuracy: 0.677151620388031


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 262ms/step
Predicted sentiment: negative
